In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver
from lxml import etree
import os
from os import path
import shutil
import time
import demjson
from selenium.webdriver.chrome.options import Options
from multiprocessing.dummy import Pool

In [2]:
def getResponse(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    return response

In [3]:
# 通过selenium获取page_text
def getHTMLBySelenium(url):
    chrome_options =Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(executable_path='C:/Users/Administrator/Desktop/chromedriver_win32/chromedriver.exe', options=chrome_options)
    browser.get(url)
    time.sleep(2)
    page_text = browser.page_source
    browser.quit()
    return page_text

In [4]:
# 有ts也有MP4

In [5]:
def getDetailM3U8(url):
    page_text = getHTMLBySelenium(url)
    tree = etree.HTML(page_text)
    m3u8Url = tree.xpath('//*[@id="program_type_1"]/div/div/@v-live')
    if len(m3u8Url) == 0:
        return ''
    return m3u8Url[0]

In [6]:
def getMp4Url(url):
    page_text = getHTMLBySelenium(url)
    #print(page_text)
    tree = etree.HTML(page_text)
    mp4Url = tree.xpath('//div[@class="videoWrap"]//video/@src')
    if len(mp4Url) == 0:
        return ''
    return mp4Url[0]

In [7]:
# http:\/\/live.finance.sina.cn\/bozhu\/1877471117\/114447?client_url=sinafinance%3A%2F%2Ftype%3D25%26program_type%3D8%26bid%3D1877471117%26cid%3D114447

In [8]:
def getNewsUrl(page):
    url = f'http://app.finance.sina.com.cn/course/index.php?callback=jsonp_1618881501936351&p=course&s=program&a=lists&format=json&extra=&page={page}&pagesize=20&live_status=&program_type=1%2C7%2C8&ordertype=&buid=&_=1618879123928'
    res = getResponse(url)
    res.encoding = 'utf-8'
    page_text = res.text
    linkList = []
    sonObj = demjson.decode(page_text[27:-14])
    for temp in sonObj['result']['data']:
        newsUrl = temp['newsUrl']
        #print(newsUrl)
        linkList.append(newsUrl)
    return linkList

In [9]:
def downloadEveryTs(url):
    allContent = getResponse(url).text
    fileLine = allContent.split('\n')
    downloadPath = 'download'
    if not os.path.exists(downloadPath):
        os.mkdir(downloadPath)
    # 通过判断文件头来确定是否是M3U8文件
    if fileLine[0] != "#EXTM3U":
        #raise BaseException(u"非M3U8的链接")
        print('非M3U8的链接')
        return
    else:
        unknow = True# 用来判断是否找到了下载地址
        for index, line in enumerate(fileLine):
            if "EXTINF" in line:
                unknow = False
                # 拼出ts片段的URL
                tsUrl = url.rsplit('/', 1)[0] + '/' + fileLine[index + 1]# i.ts在下一行
                #print(tsUrl)
                res = getResponse(tsUrl)
                cFuleName = (str(fileLine[index + 1])).split('?')[0]
                with open(downloadPath + '/' + cFuleName, 'ab') as fp:
                    fp.write(res.content)
                    fp.flush()
        if unknow:
            raise BaseException(u"未找到对应的下载地址")
        else:
            print("ts 下载完成")

In [10]:
# ts文件合并
def file_walker(path):
    file_list = []
    for root, dirs, files in os.walk(path): # 生成器
        for fn in files:
            p = str(root+'/'+fn)
            file_list.append(p)
 
    #print(file_list)
    return file_list
 
def combine(ts_path, combine_path, file_name):
    file_list = file_walker(ts_path)
    cnt = 1
    Name = f'_{cnt}'
    file_path = combine_path + file_name + Name + '.ts'
    fp = open(file_path, 'wb+')
    for i in range(len(file_list)):
        fp.write(open(file_list[i], 'rb').read())
        if (i + 1) % 500 == 0:
            fp.close()
            cnt += 1
            Name = f'_{cnt}'
            file_path = combine_path + file_name + Name + '.ts'
            fp = open(file_path, 'wb+')
    fp.close()
            
# 删除之前下载的一段段的ts文件
def clear():
    shutil.rmtree('./download')
    time.sleep(1)
    os.mkdir('download')

In [11]:
def downloadTs(url, fileName):
    downloadEveryTs(url)
    combine('./download', 'E:/work/ts/', fileName)
    print(fileName, ' successful')
    clear()

In [12]:
def downloadMp4(url, fileName):
    content = getResponse(url).content
    path = 'mp4/'
    with open(path + fileName + '.mp4', 'wb') as fp:
        fp.write(content)
    print(fileName, " successful")

In [ ]:
total = 470
for page in range(27, 101):
    print(f'page {page}')
    for link in getNewsUrl(page):
        if total < 180:
            total = total + 1
            continue
        print(link)
        m3u8Url = getDetailM3U8(link)
        if m3u8Url != '':
            downloadTs(m3u8Url, str(total))
            total = total + 1
            continue
        mp4Url = getMp4Url(link)
        if mp4Url != '':
            downloadMp4(mp4Url, str(total))
            total = total + 1
            continue
        total = total + 1
        print(str(total), ' empty')

page 27
http://live.finance.sina.cn/bozhu/3207370702/109191?client_url=sinafinance%3A%2F%2Ftype%3D25%26program_type%3D1%26bid%3D3207370702%26cid%3D109191
